In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
class k_means(object):
    def __init__(self):
        pass
    
    def fit(self):
        pass
    
    def predict(self):
        pass

In [ ]:
def sample_random_balls(n_centers, n_points, x_range = [-10, 10], y_range = [-10, 10], rotate = False):
    """
    Want to sample K blobs, with n_points per blob. Diversity range is controlled by x_range and y_range
    """
    
    x_centers = np.